In [135]:
import pandas as pd
from pprint import pprint
import pandas as pd
import plotly.graph_objects as go
import numpy as np
import datetime
from google.colab import files

In [136]:
df = pd.read_csv('/content/drive/MyDrive/data_2020_2023_btc/BTC_2019_2023_1d.csv')
df.set_index('datetime')
df = df.tail(1482) #Drop initial data

In [137]:
import os
import sys
!git clone https://github.com/ztuntrade/untrade-sdk.git
os.chdir('untrade-sdk')
!pip3 install .
untrade1= os.path.abspath('untrade-sdk')
sys.path.append(untrade1)
from untrade.client import Client
client = Client()

Cloning into 'untrade-sdk'...
remote: Enumerating objects: 118, done.
remote: Counting objects: 100% (118/118), done.
remote: Compressing objects: 100% (72/72), done.
remote: Total 118 (delta 56), reused 95 (delta 36), pack-reused 0 (from 0)
Receiving objects: 100% (118/118), 43.67 KiB | 667.00 KiB/s, done.
Resolving deltas: 100% (56/56), done.
Processing /content/untrade-sdk/untrade-sdk/untrade-sdk/untrade-sdk/untrade-sdk/untrade-sdk/untrade-sdk/untrade-sdk
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for untrade: filename=untrade-0.1.0-py3-none-any.whl size=5079 sha256=16566ba2cb84de9ead5673f93ab1907645436216ee30105c4887ae6208b033d5
  Stored in directory: /root/.cache/pip/wheels/c4/26/80/55740aa9fdaf05bd2a90882e9b030b9c03e264c0edcbda64b0
Successfully built untrade
  Attempting uninstall: untrade
    Found existing installation: untrade 0.1.0
    Uninstalling untrade-0.1.0:
      

In [138]:

#!wget http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
#!tar -xvzf ta-lib-0.4.0-src.tar.gz
#!cd ta-lib && ./configure --prefix=/usr && make && make install


In [139]:
#pip install ta-lib

In [140]:
import talib

In [141]:
def preprocessing(df):
  rsi = talib.RSI(df["close"], timeperiod=14)             #calculate necessary indicators
  ema_9 = talib.EMA(df["close"], timeperiod=9)
  ema_20 = talib.EMA(df["close"], timeperiod=20)
  sar = talib.SAR(df['high'], df['low'], acceleration=0.02, maximum=0.2)
  df['RSI'] = rsi
  df['EMA_9'] = ema_9
  df['EMA_20'] = ema_20
  df['SAR'] = sar
  ema_7_rsi =  talib.EMA(df["RSI"], timeperiod=6)
  df['EMA_7_RSI'] = ema_7_rsi
  df['p_SAR'] = df['SAR'].shift(1)
  df['p_RSI'] = df['RSI'].shift(2)
  df = df.dropna()
  return df
df = preprocessing(df)

In [142]:
def signal_gen(df):                             #signal generation
  df['signals'] = 0
  for index, row in df.iterrows():
    if row['EMA_7_RSI'] > 60:
      if row['EMA_9'] > row['EMA_20']:
        df.at[index, 'signals'] = 2
      else:
        df.at[index, 'signals'] = -2
    else:
      if row["EMA_7_RSI"]<40:
        if(row["SAR"]<=row["close"]) and (row["p_SAR"]>row["SAR"]):
          df.at[index, 'signals'] = 0
        else:
          df.at[index, 'signals'] = -1
      else:
        if row['RSI']>54 and row["p_RSI"]>row["RSI"]:
          df.at[index, 'signals'] = -1
        else:
          df.at[index, 'signals'] = 0
signal_gen(df)
df = df.drop(df.index[0])

<ipython-input-142-4a827d752b48>:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [143]:
def process(df):                                  #cleaning and filtering the signal
    df["new"] = df['signals'].copy()
    last= None
    for i in range(len(df)):
        current = df.iloc[i]["new"]
        if current != 0:
            if last is not None and (last*current>= 0):
                df.iloc[i, df.columns.get_loc("new")] = 0
            else:
                last = current
    df["signals"] = df["new"]
    df.drop(columns=["new"], inplace=True)
    j = 0;
    x  = 1
    if 'signals' in df.columns:
      for i in range(len(df)):
        if(df.iloc[i]['signals']!=0):
          if(j%2==0):
            df.at[df.index[i], 'signals'] = 1
          j=j+1
    return df
df = process(df)

In [144]:
df.to_csv('/content/Test.csv', index=False)

BACKTESTER

In [145]:
# Function to plot trade signals on a candlestick chart
def plot_signals(csv_file):
    """
    Plots a candlestick or line chart with trade signals and reversals from a CSV file.

    Parameters:
    csv_file (str): Path to the CSV file containing the data.
    """
    # Load data from the CSV file
    data = pd.read_csv(csv_file, index_col='datetime', parse_dates=True)

    # Map trade types for better visualization and debugging
    data['trade_type'] = data['signals'].map({
        1: 'Long',
        -1: 'Short',
        0: 'Hold',
        2: 'Long Reversal',
        -2: 'Short Reversal'
    })

    # Create a figure for the chart
    fig = go.Figure()

    # Plot the price data as a candlestick chart
    fig.add_trace(go.Candlestick(
        x=data.index,
        open=data['open'],
        high=data['high'],
        low=data['low'],
        close=data['close'],
        name='Candlestick',
        increasing_line_color='green',
        decreasing_line_color='red'
    ))

    # Plot Long (1) signals as green upward triangles
    fig.add_trace(go.Scatter(
        x=data.index[data['signals'] == 1],
        y=data['close'][data['signals'] == 1],
        mode='markers',
        marker=dict(symbol='triangle-up', color='lime', size=10, line=dict(color='black', width=1)),
        name='Long (1)'
    ))

    # Plot Short (-1) signals as red downward triangles
    fig.add_trace(go.Scatter(
        x=data.index[data['signals'] == -1],
        y=data['close'][data['signals'] == -1],
        mode='markers',
        marker=dict(symbol='triangle-down', color='red', size=10, line=dict(color='black', width=1)),
        name='Short (-1)'
    ))

    # Plot Long Reversal (2) signals as cyan stars
    fig.add_trace(go.Scatter(
        x=data.index[data['signals'] == 2],
        y=data['close'][data['signals'] == 2],
        mode='markers',
        marker=dict(symbol='star', color='cyan', size=10, line=dict(color='black', width=1)),
        name='Long Reversal (2)'
    ))

    # Plot Short Reversal (-2) signals as magenta stars
    fig.add_trace(go.Scatter(
        x=data.index[data['signals'] == -2],
        y=data['close'][data['signals'] == -2],
        mode='markers',
        marker=dict(symbol='star', color='magenta', size=10, line=dict(color='black', width=1)),
        name='Short Reversal (-2)'
    ))

    # Update layout to add titles and improve visualization
    fig.update_layout(
        title=f'BTC-USD Candlestick Chart with Trade Signals',
        xaxis_title='Date',
        yaxis_title='Price (USD)',
        xaxis_rangeslider_visible=False,
        legend=dict(y=1, traceorder='normal'),
        margin=dict(l=0, r=0, t=50, b=50),
    )

    # Display the chart
    fig.show()

# Function to calculate PnL and related data from a CSV file
def calculate_pnl_data(csv_file_path, initial_capital=1000):
    # Load data from the CSV file
    data = pd.read_csv(csv_file_path)

    # Initialize PnL-related columns
    data['entry'] = 0.0
    data['exit'] = 0.0
    data['quantity'] = 0.0
    data['pnl'] = 0.0
    data['Transaction_Cost'] = 0.0
    data['Net_Pnl'] = 0.0
    data['drawdown'] = 0.0
    trade_type = None
    data['trade_type'] = trade_type
    final_capital = initial_capital
    trades = 0

    # Loop through the data to find trades
    for i in range(len(data)):
        prev = data['signals'][:i].sum()
        entry = 0.0
        exit = 0.0
        quantity = 0.0
        trade_type = 'exit'

        if data.loc[i, 'signals'] != 0 or final_capital != 0:
            j = i + 1
            if data.loc[i, 'signals'] == 1 and prev == 0:  # Long entry
                while j < len(data) and (data.loc[j, 'signals'] not in [-1, -2]):
                    j += 1
                if j < len(data):
                    trades += 1
                    entry = data.loc[i, 'close']
                    exit = data.loc[j, 'close']
                    trade_type = 'long'
                    quantity = abs(final_capital / entry)
                    data.loc[i, 'quantity'] = quantity

            elif data.loc[i, 'signals'] == -1 and prev == 0:  # Short entry
                while j < len(data) and (data.loc[j, 'signals'] not in [1, 2]):
                    j += 1
                if j < len(data):
                    trades += 1
                    entry = data.loc[i, 'close']
                    exit = data.loc[j, 'close']
                    trade_type = 'short'
                    quantity = abs(final_capital / entry)
                    data.loc[i, 'quantity'] = quantity

            if j < len(data):  # Make sure we found an exit
                data.loc[i, 'entry'] = entry
                data.loc[i, 'exit'] = exit
                data.loc[i, 'quantity'] = quantity
                data.loc[i, 'trade_type'] = trade_type

                pnl = quantity * (exit - entry) if data.loc[i, 'signals'] > 0 else quantity * (entry - exit)
                transaction_cost = quantity * (entry) * 0.0015
                net_pnl = pnl - transaction_cost
                final_capital += net_pnl

                data.loc[i, 'pnl'] = pnl
                data.loc[i, 'Transaction_Cost'] = transaction_cost
                data.loc[i, 'Net_Pnl'] = net_pnl

    # Cumulative data
    data['Cumulative_PnL'] = data['pnl'].cumsum()
    data['Cumulative_Transaction_Cost'] = data['Transaction_Cost'].cumsum()
    data['Cumulative_Net_Pnl'] = data['Net_Pnl'].cumsum()

    # Calculate Buy-and-Hold and Equity curves
    buy_and_hold_start = data['close'].iloc[0]
    data['Buy_and_Hold'] = (data['close'] / buy_and_hold_start) * initial_capital
    data['Equity'] = initial_capital + data['Cumulative_Net_Pnl']

    # Calculate Drawdown
    data['Peak_Equity'] = data['Equity'].cummax()
    data['drawdown'] = (data['Equity'] - data['Peak_Equity']) / data['Peak_Equity']
    max_drawdown = data['drawdown'].min()

    # Save the updated DataFrame to a CSV file
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    output_file = f"/content/Trade_Logs_{timestamp}.csv"
    data.to_csv(output_file, index=True)

    return data, output_file

# Final function to process the trading data
def backtest(csv_file_path, initial_capital=1000, start_date=None, end_date=None):
    # Plot signals
    plot_signals(csv_file_path)

    # Calculate PnL and filter data
    data, output_file = calculate_pnl_data(csv_file_path, initial_capital)

    if start_date:
        data = data[data['datetime'] >= start_date]
    if end_date:
        data = data[data['datetime'] <= end_date]

    print(f"Output File: {output_file}")

In [146]:
def perform_backtest(csv_file_path):

    # Create an instance of the untrade client
    client = Client()

    # Perform backtest using the provided CSV file path
    result = client.backtest(
        jupyter_id="pj",  # the one you use to login to jupyter.untrade.io
        file_path=csv_file_path,
        leverage=1,  # Adjust leverage as needed
    )

    return result

if __name__ == "__main__":

    # Prompt user to upload a file in Google Colab
    # print("Please upload your CSV file:")
    # uploaded = files.upload()

    # Get the uploaded file path
    # csv_file_path = list(uploaded.keys())[0]
    # print(f"File uploaded successfully: {csv_file_path}")

    csv_file_path = '/content/Test.csv'

    # Perform backtest using the uploaded CSV file
    backtest_result = perform_backtest(csv_file_path)

    # Display the backtest result
    print("Backtest results:")
    for value in backtest_result:
        print(value)

    backtest(csv_file_path)

Backtest results:
data: {
  "jupyter_id": "pj",
  "result_type": "Main",
  "message": "Backtest completed",
  "result": {
    "static_statistics": {
      "From": "2020-01-01 00:00:00",
      "Total Trades": 21,
      "Leverage Applied": 1.0,
      "Winning Trades": 13,
      "Losing Trades": 8,
      "No. of Long Trades": 21,
      "No. of Short Trades": 0,
      "Benchmark Return(%)": 514.515732,
      "Benchmark Return(on $1000)": 5145.157324,
      "Win Rate": 61.904762,
      "Winning Streak": 5,
      "Losing Streak": 3,
      "Gross Profit": 3209.026267,
      "Net Profit": 3177.526267,
      "Average Profit": 151.310775,
      "Maximum Drawdown(%)": 8.314423,
      "Average Drawdown(%)": 2.184491,
      "Largest Win": 1752.730611,
      "Average Win": 303.716983,
      "Largest Loss": -244.001196,
      "Average Loss": -96.349313,
      "Maximum Holding Time": "106 days 0:0:0",
      "Average Holding Time": "30 days 2:17:8",
      "Maximum Adverse Excursion": 25.88764,
      "A

Output File: /content/Trade_Logs_20241111_130316.csv
